# Image recognition algorithm for Swiss bird species

## 1) The problem

Switzerland has several hundred bird species. For an amateur, identifying them is not always easy. In addition to the similarity between certain species, it is above all the large number of possibilities that discourages emerging ornithological vocations. When trying to identify a species, the amateur can look for information in books. In this case, unless he is permanently walking around with a 500-page illustrated book, he can only get them once he gets home. In addition, this research is likely to be tedious. In addition to the visual appearance, other criteria such as location or observation period determine the species of the observed bird. However, the format of a book makes it difficult to cross-reference this information.

Imagine that an amateur ornithologist has an application that allows him to instantly identify the bird he has just photographed with his smartphone. By cross-referencing the photo with the location and date of observation, the application could indicate the species to which the bird is likely to belong and a percentage of reliability. Such an application requires an algorithm to classify birds according to images and metadata. We would like to work on the development of this algorithm.

Switzerland has more than 400 species of birds. Building a classifier that takes all species into account is therefore an impossible task at my level. However, by cleverly defining the target classification group, a useful tool can be obtained. In this first project, the aim was to form small classification pools and to test different methods in order to get an idea of the achievable classification accuracy as well as the different problems that can arise.

## 2) The data

Due to various legal and organizational problems, the project was suspended between November and January. Indeed, I wanted to work with data from the Swiss Ornithological Institute through the website [ornito.ch](https://www.ornitho.ch/), which could not be delivered to me during this whole period. In early February, as the situation had still not progressed and the time factor was beginning to become critical for me, I decided to end the collaboration with these organizations and look for other sources of data.

[eBird](https://ebird.org/home) is the North American equivalent of ornitho.ch: a platform associated with the Cornell Lab of Ornithology, on which amateur ornithologists can share their observations. Luckily, it turns out that some users of this platform are present in Switzerland and share images relating to Swiss species. It also happens that eBird makes the metadata available for free download. The downside is the number of observations: instead of thousands of observations per species with ornitho.ch, I have to make do with a maximum of 300 with eBird. However, by choosing the classes well and setting smaller goals, I think I can still do something interesting. Here is the list of the species with the largest number of images, which I have selected for the EDA : 

1. [Chroicocephalus ridibundus](https://ebird.org/species/bkhgul/CH)
2. [Podiceps cristatus](https://ebird.org/species/grcgre1/CH)
3. [Netta rufina](https://ebird.org/species/recpoc/CH)
4. [Fulica atra](https://ebird.org/species/eurcoo/CH)
5. [Cygnus olor](https://ebird.org/species/mutswa/CH)
6. [Pyrrhocorax graculus](https://ebird.org/species/yebcho1/CH)
7. [Mergus merganser](https://ebird.org/species/commer/CH)
8. [Aythya fuligula](https://ebird.org/species/tufduc/CH)
9. [Larus michahellis](https://ebird.org/species/yelgul1/CH)
10. [Ardea cinerea](https://ebird.org/species/graher1/CH)
11. [Buteo buteo](https://ebird.org/species/combuz1/CH)
12. [Milvus milvus](https://ebird.org/species/redkit1/CH)
13. [Turdus merula](https://ebird.org/species/eurbla/CH)
14. [Aquila chrysaetos](https://ebird.org/species/goleag/CH)

However, the quality of the metadata is better than with ornitho.ch. The dataset includes 56 variables ranging from date to specimen behavior and geographic coordinates. Most of these variables do not include enough observations to be useful, but the variables I intended to use with ornitho.ch are all present and do not contain Nan.

### First classification pool

Given the small number of observations, we must be humble and avoid setting irrealistic goals. A classification task with 5 classes that are relatively homogenous within each class and relatively heterogenous between them is a reasonable challenge. Based on the results of exploratory metadata analysis and the visual assessment of the pictures, I have drawn the following conclusions.

- Aquila chrisaetos and Turdus merula can be eliminated since we don't have enough observations.
- Mergus merganser has a pronounced sexual dimorphism. In addition, the female looks pretty much like Podiceps cristatus. As a result, female Mergus merganser are highly likely to be misclassified as Podiceps cristatus. For this reason, I rule out Mergus merganser.
- For Aythya fuligula, Cygnus olor, Fulica atra, Netta rufina, Podiceps cristatus, and to a lesser extent Larus michahellis and Chroicocephalus ridibundus, the background is mostly water. This may make the images very similar for the algorithm. Nevertheless, it can be reduced by cropping the images close to the bird.
- The same is true with respect to the blue sky for Buteo buteo, Milvus milvus, Pyrrhocorax graculus and to a lesser extent Larus michahellis and Chroicocephalus ridibundus.
- Species with a larger number of observations should be preferred ; we can always drop observations if necessary.
- All species still have some images that are not appropriate for classification: immature specimens, very distant group of birds, etc. However, these images are few and it will be easy to spot them visually and drop them from the dataset.

I selected the following species in a first classification pool:

1. Podiceps cristatus
2. Fulica atra
3. Chroicocephalus ridibundus
4. Cygnus olor
5. Pyrrhocorax graculus

### Second classification pool

In a second phase, I would like to increase the difficult a little by introducing species that looks like other species. Indeed, I want to replace Chroicocephalus ridibundus by Netta Rufina which quite similar in colors to Podiceps cristatus, and replace Cygnus olor by Turdus merula which is very similar to Pyrrhocorax graculus and has the same dominant color as Fulica atra.

In particular, it will be interesting to see if a can settle the algorithm in a way to give sufficiently weight to the metadata to separate similar looking species. Moreover, Netta rufina has a quite strong sexual dimorphism, but the male and female are likely to be observed in the same places at the same period.

We also note that Turdus merula has far less observations than other classes. In would be interesting to see what is the effect on the classification and which strategies we can find to handle potential problems.

The second classification contains the following species:

1. Podiceps cristatus
2. Fulica atra
3. Netta rufina
4. Turdus merula
5. Pyrrhocorax graculus

### 3) EDA and Data engineering

The EDA suggests that the month in which the photo was taken is a predictor that can make a useful contribution to classification. This is certainly due to the fact that many species are migratory. Location also appears to be a good predictor, as some species have well-defined ranges. Neither of these two predictors alone can differentiate between species, but they are useful in combination with other predictors, especially in distinguishing species with a very similar appearance. 

The other variables available on eBird, on the other hand, are not useful for classification, either because their distribution is similar for most species or because the number of NaN is too high to make them usable. However, I have used the non-null values of some variables to systematically remove images that are likely to be problematic for classification, for example those explicitly labeled as immature specimens or having another species in the background. I also tried to identify blurred images by converting them into grayscale and then calculating the [global laplacian variance](https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/), assuming that a blurred image will have a low value. This method has worked quite well, but a low value also characterizes images for which a uniform background (typically the sky) occupies a large part of the image. On the other hand, a particularly high value characterizes images in which the bird is present in front of a complex background (typically dense vegetation). This method could therefore also be used to discriminate images that make classification difficult because a very low variance often corresponds to images where the bird is barely visible in the sky while a very high variance corresponds to backgrounds so complex that the pixels representing the bird are "drowned" in a huge amount of noise.

Altitude is the best example of a variable that would have been useful for classification but which is not exploitable since it contains almost only NaN. Indeed, the location is indicated in the form of WGS84 coordinates, which makes it possible to calculate the distance between two observations, but does not contain information relating to the relief. However, some mountain species can be observed both in Graubünden and in Les Diablerets, but not certainly in Vevey, even though these two last places would be close in terms of horizontal distance. It is clear that by using a KNN taking into account the WGS84 coordinates but omitting the altitude, two observations made respectively in Vevey and Les Diablerets will be considered as similar while the probability that it is the same species is low. For this reason, I transformed the WGS84 coordinates into coordinates of the Swiss topographic system in order to be able to extract the altitude information using a topographic model of Switzerland.

The algorithm remains mainly based on bird images. The first problem to address was to choose the type of method I wanted to use for the classification. Having a limited number of observations and especially a limited computing power, the use of a CNN was not an option. I opted for a two-step solution: I first extracted features from the images using the model [MobileNetV2](https://keras.io/api/applications/mobilenet/), and then I used these features as predictors in models with less computing power. Upstream, I chose to standardize the image format in order to limit the computing power required and to simplify the processing by the primary algorithm. To do this, the images must be cropped to a square and resized. While resizing can be done with a very simple function, the cropping algorithm poses many problems and has a considerable impact on the final result of the classification. However, as this step was not the core of this first project, I chose not to spend too much time on it and to settle for a relatively rudimentary cropping algorithm found on this [repository](https://github.com/epixelic/python-smart-crop).

Once the images were cropped, standardized and features extracted using MobileNetV2, I selected the principal components using a PCA. I then used these features as well as metadata variables to train and tune the hyperparameters of several classifiers: KNN, logistic regression, random forest, SVM with linear and RBF kernel, fully connected neural network. The variables resulting from the metadata are the following: months (binary variables), altitude (standardized), coordinates of the Swiss topographic system (standardized with the same scaler and refocused to keep the proportions). The best classifiers achieve a performance of about 95% of correctly classified images. More than by the absolute value of the performance, I was mainly interested in determining if recurrences can be discerned among misclassified pictures, in order to identify the weaknesses of this first basic algorithm and to think about ways to improve it.

## Misclassification analysis

Metadata alone can provide significantly higher accuracy than the DummyClassifier and can differentiate fairly well between species in different environments. Pyrrhocorax graculus, present in the Alps at high altitudes, is rarely confused with other birds. However, this is not sufficient to differentiate between lake species, which all occur in similar areas and at similar altitudes. The distribution of observations over the year is not decisive in the classification because the classes do not present sufficiently differentiated profiles. This is due to the fact that the species concerned do not have a sufficiently pronounced migratory behaviour, that the number of observations is too small to be statistically significant and that the timing of the observation depends as much on the presence of the bird as on that of the observer.

The features extracted from the images are therefore necessary to achieve high classification accuracy. By inspecting the images that are misclassified, we can see that they often have particular characteristics that fall into one of the following 10 categories :

1. Juveniles have a very different appearance from adults and are often systematically misclassified.
> Ex: ML171328621 is a juvenile Fulica atra misclassified as Podiceps cristatus.
2. Some species show strong sexual dimorphism and one sex is less resembling the other than to another species. In pool 2, this is above all the case of the female Netta rufina which is very often misclassified as Podiceps cristatus. The female Turdus merula also differs slightly from the male, but does not resemble other species.
> Ex: ML155093561 is a female Netta rufina misclassified as Podiceps cristatus
3. Specimens with an unusual colour for the species are confused with another species whose usual colour more closely resembles the colour of the specimen.
> Ex : ML187212231 is a grey Cygnus olor misclassified as Podiceps cristatus / ML173059761 is a white-spotted Fulica atra misclassified as Chroicocephalus ridibundus
4. Specimens photographed in a position unusual for the species are mistaken for another species for which the position is common. 
> Ex ML189776781 is a Cygnus olor in flight misclassified as Chroicocephalus ridibundus.
5. Some images contain multiple species and are misclassified with high probability.
> Ex: ML166553541 is labelled as Fulica atra and misclassified as Cygnus olor, but the image contains both species / ML112470111 is labelled as Fulica atra and misclassified as Podiceps cristatus. The image contains no Podiceps cristatus but 4 Fulica atra, a male Netta rufina and a female.
6. Some birds are photographed at too great a distance to be identifiable from the background and are systematically misclassified.
> Ex: ML38070711 is labelled Netta rufina and misclassified as Fulica atra; in the picture only boats can be seen / ML198644951 is labelled Fulica atra and misclassified as Netta rufina; in the picture only a large group of dark birds can be seen at 100m.
7. In some images, the bird is close enough to be recognized visually, but still occupies only a small portion of the image relative to the background. Misclassifications between lake species then occur. Even though the background is almost always a water surface, it is possible that certain light levels are more randomly present for certain species in the dataset and that this may lead to misclassification. In this case, the problem is that the pixels in the background play a greater role in the classification than the pixels of the bird itself.
> ML54145111 is a Fulica atra misclassified as Podiceps cristatus / ML114926971 is a Netta rufina misclassified as Podiceps cristatus / ML106693951 is a Podiceps cristatus misclassified as Netta rufina
8. Specimens present in an environment more common for another species are regularly misclassified.
> Ex: ML167856591 is a Turdus merula in flight in front of rocks and misclassified as Pyrrhocorax graculus / ML176744871 is a Turdus merula photographed at the water's edge in front of reeds and misclassified as Fulica atra / ML41377851 is a Fulica atra photographed in green vegetation and misclassified as Turdus merula.
9. For some photos, an unusual background induces a misclassification even if it does not correspond to an environment common to another species.
> Ex: ML127044901 is a Turdus merula photographed on a gutter and misclassified as Netta rufina.
10. Some photos are taken at a reasonable distance, but are cropped incorrectly by the algorithm, so that the bird does not appear or not entirely.
> Ex: ML200196631 is a Chroicocephalus ridibundus misclassified as Pyrrhocorax graculus. On the picture, we can only see the statue on which the bird is.

![misclassification_analysis](data/other/misclassification_analysis.png)

## Solutions by data collection

Many of the problems mentioned above can be solved by better labelling of observations. In the original dataset, some variables that can solve these problems exist but very few values are entered. Provided we have access to it, we can construct the user data entry process in such a way that these values must be entered to submit an image. In the context of an application, these mandatory labeling steps can take the form of a list of predefined choices, which limits the effort required of the user and ensures that there are categories that can be used to filter the observations. Having these categories available would significantly increase the accuracy of classification. However, care must be taken not to impose too many mandatory steps on users, otherwise the process would be tedious and discourage data entry. 

In particular, the following problems can be solved by better labelling:

- Problems 1 and 2 can be solved by introducing a mandatory labeling step that asks the user to categorize the observation as male, female or juvenile. A further distinction can be made between fledglings and juveniles when the latter differ from adults in plumage, for example, as in the case of Larus Michaellis. Categories can be defined for each species according to its specificities for greater precision or for all species for simplicity. Juveniles may be considered as an additional class if their numbers are sufficient or excluded otherwise. Considering males and females with strong sexual dimorphism as different classes should help classification, but it may also create an imbalance in classes that needs to be compensated for.
- Problem 3 can be solved by introducing a mandatory labelling step that requires the user to mark specimens with particular characteristics. Input options can typically be limited by a species-specific pick-list when appearance variants are common (e.g. Grey Swan) in order to obtain more easily exploitable values. In general, however, the most important thing is to separate special specimens from the mass of normal specimens. Unless very large amounts of data are available, special specimens cannot constitute balanced classes, and they will simply be excluded.
- Problem 4 can be partially solved by introducing a mandatory labeling step that asks the user to specify the type of shot. In order to obtain usable results, it is important here to limit the choices to a species-specific list (e.g. for Cygnus olor: in flight, on the ground, on the water, bird group, macro, special position). If these categories are large enough, they may constitute classes, but it is more likely to exclude unusual shots.
- Problem 5 can be solved by introducing a mandatory labeling step that asks users to report the different species present in the photo. This labeling can be refined by specifying the number of specimens of each species but this can quickly become tedious and the main thing is to identify images containing several species. An additional possbility is to ask the user to frame the birds and enter them as separate observations.
- Problems 6 to 10 can be solved by asking the user to frame the birds in the picture or to report if this is not possible (too far away, too large a group). Better framing of birds avoids errors due to the presence of several birds, the absence of recognizable birds, and unusual background, but may increase errors due to the physical similarity between some species since the background will be less able to differentiate between them. In pool 2, this mainly concerns Pyhrrocorax graculus and Turdus merula, which are present in rather different environments. However, the benefit seems to be more important than the loss, because the errors thus avoided are numerous and the two similar species can be differentiated by the metadata.

These solutions avoid having to do a lot of data processing work, but require access to the data entry process, for example through our own platform or application. If we simply retrieve data collected by third parties, other solutions are necessary to try to retrieve the same information.

## Solutions by data processing

In the event that we do not have access to the user input process, we may implement the following strategies to attempt to collect the same information:

- A better cropping of the photos directly solves problems 7-10 and indirectly solves problems 5-6. It is worth developing a powerful cropping algorithm that accurately selects specimens. Rather than using a fairly rudimentary method that detects interesting features in photos, a promising approach is to use an algorithm specifically trained to recognize birds. All birds share a number of common characteristics, and this allows the specimen to be separated from the background and other objects.
    - In the case of problem 6, however, it is possible that the algorithm does not find a bird and selects another salient object. However, these cases are limited and an improvement is likely even for this problem. 
    - In the case of Problem 5, the presence of a very large number of birds can lead to the same situation, or to the fact that the cropped images have too few pixels to be usable. However, it is sufficient to simply eliminate images with less than 224 pixels by 224.
    - On the other hand, applying the cropping algorithm to images with too many pixels can be very expensive in terms of computing power. It may be useful to decrease the resolution of very large images beforehand.
    - Finally, the cropping algorithm may not define a square frame. If we want to keep the functions used here that require square images, the best solution seems to me to adapt the cropping algorithm so that it provides the coordinates of the frame in the original image, but without eliminating the rest. We could then use these coordinates to define the best square frame of the specimen.
- The 10 problems identified result in the features of misclassified images being far away from the features of other images of this species. By displaying the 2 principal components of the features extracted from the images, we can see that some observations are far from the mass of observations of this species. We could try to calculate the center of the cloud of observations for the n principal components, then define a threshold of distance from this center beyond which the observations are considered as outliers. However, it is almost impossible to find the right threshold other than by trials, and the method as a whole remains rather rough. This selection should be learned by the algorithm and not done manually . However, it would be interesting to display a representation of the two or three principal components after a precise cropping with the above mentioned algorithm. The species should then be more distinct on the representation
- An interesting approach for problems 1-4 would be to use an unsupervised algorithm only on features of the same species to isolate special specimens and separate the male and female Netta rufina.
- Problems 4, 8 and 9 can be addressed by defining the average colour of each species to identify images with unusual colours. If an efficient cropping algorithm is used beforehand, only problem 4 is really important and this solution seems promising in this case.
- The variance of the laplacian can be used to identify images with a uniform (low value) or complex (high value) background. This makes it possible to identify problematic images for problems 6 and 7, as long as the background is uniform or complex. However, this method is not as good as a precise cropping algorithm because it does not produce a better image of the specimen. The best we can do with spotted problematic images is to exclude them from features. One can also use the variance of the laplacian for problems 8 and 9 by displaying its distribution to spot outliers. The involved images can then be excluded. With respect to Problem 5, it is also possible that a high variance of the laplacian may reveal a high number of specimens if those contrast with the background.

![pac2](data/other/pca2_plot.png)

## Perspectives

This project can be seen as a first step in the development of a more powerful classification algorithm for a coherent group of birds. Through my work, I was able to get an idea of the possibilities in this field and identify areas for improvement. In the future, I would like to develop this project in the following steps:

1. Test improvement options on the same data in order to assess their potential.
2. Define a coherent group of birds of interest to potential users. This group will typically be larger than the pools defined in this project, which increases the difficulty of classifying ceteris paribus and possibly calls for the use of more observations and other models. If I decide to develop an application, a pool containing common lake species seems a tempting idea. These are indeed easily observable by potential non-specialist users. Moreover, the number of these species is large enough to represent a difficulty for the beginner while constituting an ambitious but feasible classification task.
3. Build a structure for the online storage and operation of the algorithm and data.
4. Create an interface for users to enter data and obtain a prediction.
5. Develop a process to improve the performance of the algorithm with the data transmitted by users.
6. Address legal issues.

These are the perspectives that I had sketched out at the end of this first project. However, it is a very practical project, applying already proven methods to a concrete problem, so I am not convinced that it meets the criteria of an academic work. However, it might be possible to focus on one aspect of the project and develop an ad hoc method to solve one of the problems identified in this first project. Some possible approaches could be :

- Developing a more efficient cropping algorithm
- Develop an algorithm to better label the images:
    - Train a classifier to separate specimens in flight, floating or on land
    - Develop an algorithm to discriminate between images that are too uniform (empty sky or water body) or too complex (large group of birds, dense vegetation) using Signal Processing methods.
    - Train a classifier to separate images containing a single bird from images containing several birds.
- Programming a classifier from a to z (without using an ML library) but focusing on a reduced classification pool (even if it is a bit like reinventing the wheel...)
- Renounce the use of metadata and train a ConvNet on a much larger number of observations (without metadata, one can use the images available on eBird for specimens observed outside Switzerland) by class (but the computing power remains an important limiting factor...)
